### 

### Print Spark Version

In [1]:
spark.version

Starting Spark application


SparkSession available as 'spark'.
u'2.2.0.2.6.3.0-235'

## Sample WordCount Job

In [2]:
lines = sc.parallelize(['Its fun to have fun,','but you have to know how.']) 
wordcounts = lines.map( lambda x: x.replace(',',' ').replace('.',' ').replace('-',' ').lower()) \
        .flatMap(lambda x: x.split()) \
        .map(lambda x: (x, 1)) \
        .reduceByKey(lambda x,y:x+y) \
        .map(lambda x:(x[1],x[0])) 

wordcounts.collect()



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[(2, 'have'), (1, 'but'), (2, 'to'), (1, 'you'), (1, 'how'), (1, 'know'), (1, 'its'), (2, 'fun')]

### Sample read write to Bigquery using spark  Reference: https://github.com/GoogleCloudDataproc/spark-bigquery-connector 

In [4]:
bq_query = """select * from `wmt-mlp-prod.workshop1.table1`"""
df = spark.read.format("bigquery").option('parentProject', 'wmt-mlp-prod') \
      .option('viewMaterializationProject','wmt-mlp-prod') \
      .option('materializationDataset','workshop1') \
      .option('project', 'wmt-mlp-prod') \
      .option('viewsenabled','true') \
      .load(bq_query)
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+------+-----+
|  name|degree|score|
+------+------+-----+
|pankaj|   BCA|   40|
| Geeku|   MBA|   98|
|aparna|   MBA|   90|
|sudhir|M.Tech|   80|
+------+------+-----+

In [6]:
df.write \
  .format("bigquery") \
  .option("temporaryGcsBucket","element-workshop-1") \
  .save("wmt-mlp-prod:workshop1.table2")

students_df = spark.read.format('bigquery') \
  .option('table', 'wmt-mlp-prod:workshop1.table2') \
  .load()
students_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+------+-----+
|  name|degree|score|
+------+------+-----+
|pankaj|   BCA|   40|
| Geeku|   MBA|   98|
|aparna|   MBA|   90|
|sudhir|M.Tech|   80|
+------+------+-----+

### Sample read write to GCS using spark

In [7]:
dfWrite = spark.createDataFrame([
    (0, "sachin0", "M", "Math"),
    (1, "sachin1", "M", "Chemistry"),
    (2, "sachin2", "M", "Physics"),
    (3, "sachin3", "M", "English"),
    (4, "sachin4", "M", "Hindi"),
    (5, "sachin5", "M", "Computer")
], ["id", "Name", "Gender", "Subject"])

dfWrite.show()



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+-------+------+---------+
| id|   Name|Gender|  Subject|
+---+-------+------+---------+
|  0|sachin0|     M|     Math|
|  1|sachin1|     M|Chemistry|
|  2|sachin2|     M|  Physics|
|  3|sachin3|     M|  English|
|  4|sachin4|     M|    Hindi|
|  5|sachin5|     M| Computer|
+---+-------+------+---------+

In [12]:
bucket_name = "element-workshop-1"
file_path = 'data/pyspark_df_to_gcs.csv'
abs_path = f'gs://{bucket_name}/{file_path}'
print(abs_path)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

gs://element-workshop-1/data/pyspark_df_to_gcs.csv

In [13]:
dfWrite.write.mode("overwrite").option("header",True).csv(abs_path)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
dfRead = spark.read.option("header",True).csv(abs_path)
dfRead.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+-------+------+---------+
| id|   Name|Gender|  Subject|
+---+-------+------+---------+
|  1|sachin1|     M|Chemistry|
|  2|sachin2|     M|  Physics|
|  4|sachin4|     M|    Hindi|
|  5|sachin5|     M| Computer|
|  3|sachin3|     M|  English|
|  0|sachin0|     M|     Math|
+---+-------+------+---------+

## Sample read write from BFD Hive Metastore

In [1]:
sqlContext.sql("show databases").show(10,False);

Starting Spark application


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+
|databaseName|
+------------+
|a0a0an1     |
|a0g06lc     |
|a0m0y7x     |
|a0s023t     |
|a0s06uq     |
|a0s0e0u     |
|a0s0rxz     |
|abc         |
|abc1        |
|abc11       |
+------------+
only showing top 10 rows

In [ ]:
spark.sql("show tables in default").show(10,False);
# spark.catalog.listTables("default");

In [12]:
firstTable=spark.sql("show tables in default").first()[1]
print("tableName =" + firstTable)
sql="select * from {} limit 10".format(firstTable);
print( "sql = " +sql)
print( "*******************")
# spark.catalog.listTables("default");
spark.sql(sql).show(10,False);

tableName =1po_1rcv
sql = select * from 1po_1rcv limit 10
*******************
+--------+----------+------------+------+----------+----------+-----------+--------------+-----------------+-----------------+------------+-----------------+----------------+---------------------+---------------------+------------------------+-----------------+--------------------+-----------------+----------------+------------+---------+--------+-------+----------+-----------+----------+
|load_id |trailer_id|receiver_nbr|dc_nbr|po_nbr    |order_date|op_cmpny_cd|bill_type_code|snapshot_beg_date|snapshot_end_date|orign_loc_id|orign_loc_type_cd|est_tot_cost_amt|actual_total_cost_amt|est_linehaul_cost_amt|actual_linehaul_cost_amt|est_accs_cost_amt|actual_accs_cost_amt|est_stop_chrg_amt|tot_distance_qty|distance_qty|tms_cases|tms_cube|tms_wgt|tms_pallet|dest_loc_id|load_date |
+--------+----------+------------+------+----------+----------+-----------+--------------+-----------------+-----------------+------------